# Proyek Analisis Data: E-Commerce Public
- **Nama:** Samuel Pandohan Terampil Gultom
- **Email:** pandohansamuel19@gmail.com
- **ID Dicoding:** pandohansamuel19

## Menentukan Pertanyaan Bisnis

- Dalam 3 bulan terakhir, kategori produk apa yang paling sering dibeli oleh pelanggan baru, dan bagaimana kontribusinya terhadap total pendapatan?
- Apa 5 wilayah pengiriman dengan waktu pengiriman rata-rata tercepat dan terlambat dalam 6 bulan terakhir, dan bagaimana hal ini memengaruhi tingkat kepuasan pelanggan berdasarkan ulasan?
- Selama 1 tahun terakhir, kategori produk apa yang memiliki tingkat pengembalian (return rate) tertinggi, dan bagaimana pola ulasan pada produk-produk tersebut?
- Siapa 10 penjual dengan rata-rata nilai ulasan terbaik dan terburuk selama 12 bulan terakhir, dan bagaimana performa mereka dalam hal jumlah pesanan dan pendapatan?
- Metode pembayaran mana yang paling banyak digunakan dalam 6 bulan terakhir, dan bagaimana distribusinya terhadap nilai pesanan (order value) di setiap kategori produk?

## Import Semua Packages/Library yang Digunakan

In [72]:
from pathlib import Path
from functools import reduce
from typing import List, DefaultDict
from dataclasses import dataclass
import os
import pandas as pd
import numpy as np
import seaborn as sns
import json

Path.cwd()

PosixPath('/home/adalgiso/self/nn/supervised/ml-development/simple-eda')

## Data Wrangling

### Gathering Data

#### Import All Raw Data

In [41]:
raw_dt = {
    "orders": pd.read_csv("data/orders_dataset.csv"),
    "sellers": pd.read_csv("data/sellers_dataset.csv"),
    "products": pd.read_csv("data/products_dataset.csv"),
    "customers": pd.read_csv("data/customers_dataset.csv"),
    "geolocation": pd.read_csv("data/geolocation_dataset.csv"),
    "order_items": pd.read_csv("data/order_items_dataset.csv"),
    "order_payments": pd.read_csv("data/order_payments_dataset.csv"),
    "product_reviews": pd.read_csv("data/order_reviews_dataset.csv"),
    "product_category": pd.read_csv("data/product_category_name_translation.csv"),
}

In [3]:
raw_dt["orders"].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [8]:
raw_dt["order_items"].head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [10]:
raw_dt["products"].head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [11]:
raw_dt["product_category"].head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [12]:
raw_dt["order_payments"].head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [13]:
raw_dt["product_reviews"].head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


**Insight:**
- In Total, there's 9 Raw Data for Brazilian E-Commerce Public dataset with its informations
- In Order, there's 3 most relational dataset in its keys to others, which are **orders_dataset**, **order_items_dataset**, and **products_dataset**

### Assessing Data

#### Check all data types on each raw data

In [44]:
for name, types in raw_dt.items():
    print(types.info(), "\n")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   

#### Update Timestamp

In [61]:
ts_orders_columns = [
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date",
    "order_estimated_delivery_date"]
ts_items_column = ["shipping_limit_date"]
ts_reviews_columns = ["review_creation_date", "review_answer_timestamp"]

for timestamp in ts_orders_columns: raw_dt["orders"][timestamp] = pd.to_datetime(raw_dt["orders"][timestamp], errors='coerce')
for timestamp in ts_items_column: raw_dt["order_items"][timestamp] = pd.to_datetime(raw_dt["order_items"][timestamp], errors='coerce')
for timestamp in ts_reviews_columns: raw_dt["product_reviews"][timestamp] = pd.to_datetime(raw_dt["product_reviews"][timestamp], errors='coerce')

raw_dt["orders"].info()
raw_dt["order_items"].info()
raw_dt["product_reviews"].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype     

#### Check Duplicate Value on each Raw Data

In [46]:
total_samples_dt = {}
each_total_duplicate_dt = {}

# Calculate and store the duplicate counts and total sample counts for each DataFrame
for df_name, df in raw_dt.items():
    total_samples_dt[df_name] = len(df)
    each_total_duplicate_dt[df_name] = df.duplicated().sum()

# Create DataFrames from the dictionaries
total_sample_counts_df = pd.DataFrame.from_dict(total_samples_dt, orient='index', columns=['Total Sample Count'])
duplicate_counts_df = pd.DataFrame.from_dict(each_total_duplicate_dt, orient='index', columns=['Duplicate Count'])

# Combine the two DataFrames by concatenating them horizontally
pd.concat([total_sample_counts_df, duplicate_counts_df], axis=1)

,Total Sample Count,Duplicate Count
orders,99441,0
sellers,3095,0
products,32951,0
customers,99441,0
geolocation,1000163,261831
order_items,112650,0
order_payments,103886,0
product_reviews,99224,0
product_category,71,0


#### Check Missing Value on each Raw Data

In [52]:
missing_values = {
    dataset_name: [
        {"column": col, "missing_values": count}
        for col, count in dataset.isnull().sum().items() if count > 0
    ]
    for dataset_name, dataset in {
        "orders": raw_dt["orders"],
        "payments": raw_dt["order_payments"],
        "order_items": raw_dt["order_items"],
        "products": raw_dt["products"],
        "products_category": raw_dt["product_category"],
        "sellers": raw_dt["sellers"],
        "reviews": raw_dt["product_reviews"]
    }.items()
}

# Convert missing values summary to a DataFrame
missing_values_df = pd.DataFrame.from_dict({
    key: {"Columns with NULL": len(value), "Total NULL": sum([v["missing_values"] for v in value])}
    for key, value in missing_values.items()
}, orient='index')

print("Missing Values Summary:")
missing_values_df

Missing Values Summary:


,Columns with NULL,Total NULL
orders,3,4908
payments,0,0
order_items,0,0
products,8,2448
products_category,0,0
sellers,0,0
reviews,2,145903


**Insight:**
- `geolocation` have most duplicate data with total `261831` datas.
- `orders`, `products`, `product_reviews` have many of duplicate with its quantities

### Cleaning Data

Melihat dari sumber data ini, kita akan mencoba melihat seberapa banyak informasi relasi pada setiap kolom pada antar dataset yang ada.
<img src="assets/e-commerce-relational.png" width="500" height="500">

`orders_id` merupakan feature/column yang memiliki relational yang paling banyak pada dataset lainnya. Selanjutnya kita akan lanjut untuk proses cleansing, terkhusus pada data yang memiliki duplikasi dan missing value.

In [66]:
raw_dt["orders"]["order_purchase_timestamp"].max()

Timestamp('2018-10-17 17:30:18')

**Insight:**
- xxx
- xxx

## Exploratory Data Analysis (EDA)

In [67]:
latest_timestamps = {
    name_dt: [
        {"column": col, "missing_values": count}
        for col, count in dataset.
    ]
    for name_dt, dt in {
        "orders": raw_dt["orders"],
        "order_items": raw_dt["order_items"],
        "reviews": raw_dt["product_reviews"]
    }.items()
}

latest_timestamps_df = pd.DataFrame.from_dict(latest_timestamp_dt, orient='index', columns=['latest_timestamp'])
latest_timestamps_df

SyntaxError: invalid syntax (3566169350.py, line 2)

In [ ]:
@dataclass
class Config:
    six_months_ago = datetime.now() - timedelta(days=180)
    one_year_ago = datetime.now() - timedelta(days=365)

### Explore analysis based on relation for define problems solution

**Insight:**
- xxx
- xxx

## Visualization & Explanatory Analysis

### Pertanyaan 1: New Customer Analysis

### Pertanyaan 2: Shipping Analysis

**Insight:**
- xxx
- xxx

### Pertanyaan 3: Product Return Analysis

### Pertanyaan 4: Seller Performance Analysis

### Pertanyaan 5: Payment Analysis

## Analisis Lanjutan (Opsional)

## Conclusion

- Conclution pertanyaan 1
- Conclution pertanyaan 2

## Berikut adalah langkah-langkah untuk menjawab **pertanyaan 1**
> *"Dalam 3 bulan terakhir, kategori produk apa yang paling sering dibeli oleh pelanggan baru, dan bagaimana kontribusinya terhadap total pendapatan?"*

---

### **1. Persiapan Data**
1. **Pahami dataset yang dibutuhkan:**
   - `customers_dataset.csv` untuk mengidentifikasi pelanggan baru.
   - `orders_dataset.csv` untuk mendapatkan tanggal pesanan.
   - `order_items_dataset.csv` untuk detail produk dalam pesanan.
   - `products_dataset.csv` untuk informasi produk.
   - `product_category_name_translation.csv` untuk terjemahan kategori produk.
   - `order_payments_dataset.csv` untuk menghitung kontribusi pendapatan.

2. **Import dataset:**
   Gunakan Python (pandas) atau SQL untuk mengimpor data. Misalnya:
   ```python
   import pandas as pd
   customers = pd.read_csv("customers_dataset.csv")
   orders = pd.read_csv("orders_dataset.csv")
   order_items = pd.read_csv("order_items_dataset.csv")
   products = pd.read_csv("products_dataset.csv")
   translations = pd.read_csv("product_category_name_translation.csv")
   payments = pd.read_csv("order_payments_dataset.csv")
   ```

---

### **2. Identifikasi Pelanggan Baru**
1. Tentukan periode waktu analisis, misalnya 3 bulan terakhir dari dataset.
   ```python
   recent_period_start = "2024-09-01"
   recent_period_end = "2024-11-30"
   ```

2. Gabungkan data **`customers_dataset`** dengan **`orders_dataset`** untuk melihat kapan pelanggan pertama kali melakukan pesanan.
   ```python
   first_orders = orders.groupby('customer_id')['order_purchase_timestamp'].min().reset_index()
   first_orders['is_new_customer'] = (first_orders['order_purchase_timestamp'] >= recent_period_start) & \
                                     (first_orders['order_purchase_timestamp'] <= recent_period_end)
   ```

3. Filter hanya pelanggan baru.
   ```python
   new_customers = first_orders[first_orders['is_new_customer']]
   ```

---

### **3. Identifikasi Produk yang Dibeli oleh Pelanggan Baru**
1. Hubungkan **pesanan pelanggan baru** dengan **order_items**.
   ```python
   new_customer_orders = orders[orders['customer_id'].isin(new_customers['customer_id'])]
   new_customer_items = order_items[order_items['order_id'].isin(new_customer_orders['order_id'])]
   ```

2. Tambahkan detail produk dari **products_dataset**.
   ```python
   new_customer_items = new_customer_items.merge(products, on='product_id', how='left')
   new_customer_items = new_customer_items.merge(translations, on='product_category_name', how='left')
   ```

---

### **4. Analisis Kategori Produk**
1. Hitung jumlah produk per kategori yang dibeli oleh pelanggan baru.
   ```python
   category_counts = new_customer_items.groupby('product_category_name_english')['order_id'].count().reset_index()
   category_counts.columns = ['category', 'num_orders']
   ```

2. Urutkan kategori berdasarkan jumlah pesanan terbanyak.
   ```python
   category_counts = category_counts.sort_values('num_orders', ascending=False)
   ```

---

### **5. Hitung Kontribusi terhadap Pendapatan**
1. Hubungkan data **order_payments** untuk menghitung total pendapatan.
   ```python
   new_customer_revenue = new_customer_items.merge(payments, on='order_id', how='left')
   revenue_per_category = new_customer_revenue.groupby('product_category_name_english')['payment_value'].sum().reset_index()
   revenue_per_category.columns = ['category', 'total_revenue']
   ```

2. Gabungkan jumlah pesanan dan pendapatan untuk setiap kategori.
   ```python
   final_result = category_counts.merge(revenue_per_category, on='category', how='left')
   ```

3. Hitung kontribusi setiap kategori terhadap total pendapatan.
   ```python
   total_revenue = final_result['total_revenue'].sum()
   final_result['revenue_contribution'] = final_result['total_revenue'] / total_revenue * 100
   ```

---

### **6. Visualisasi Hasil**
1. Gunakan visualisasi untuk menampilkan hasil.
   ```python
   import matplotlib.pyplot as plt
   
   # Bar chart jumlah pesanan
   plt.bar(final_result['category'], final_result['num_orders'])
   plt.title('Top Categories by Orders (New Customers)')
   plt.xticks(rotation=45)
   plt.show()

   # Pie chart kontribusi pendapatan
   plt.pie(final_result['revenue_contribution'], labels=final_result['category'], autopct='%1.1f%%')
   plt.title('Revenue Contribution by Category')
   plt.show()
   ```

---

### **7. Interpretasi Hasil**
Setelah mendapatkan hasil:
- Identifikasi **kategori dengan pesanan terbanyak**.
- Bandingkan dengan **kontribusi pendapatan** untuk mengetahui kategori yang paling menguntungkan.
- Berikan rekomendasi, seperti memfokuskan promosi atau meningkatkan stok pada kategori yang paling populer dan menguntungkan.

---

Apakah Anda ingin bantuan dalam mengimplementasikan salah satu langkah di atas?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Import datasets
customers = pd.read_csv("customers_dataset.csv")
orders = pd.read_csv("orders_dataset.csv")
order_items = pd.read_csv("order_items_dataset.csv")
products = pd.read_csv("products_dataset.csv")
translations = pd.read_csv("product_category_name_translation.csv")
payments = pd.read_csv("order_payments_dataset.csv")

# Step 2: Define recent period for analysis
recent_period_start = "2024-09-01"
recent_period_end = "2024-11-30"

# Step 3: Identify new customers
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
first_orders = orders.groupby('customer_id')['order_purchase_timestamp'].min().reset_index()
first_orders['is_new_customer'] = (first_orders['order_purchase_timestamp'] >= recent_period_start) & \
                                  (first_orders['order_purchase_timestamp'] <= recent_period_end)
new_customers = first_orders[first_orders['is_new_customer']]

# Step 4: Identify products bought by new customers
new_customer_orders = orders[orders['customer_id'].isin(new_customers['customer_id'])]
new_customer_items = order_items[order_items['order_id'].isin(new_customer_orders['order_id'])]
new_customer_items = new_customer_items.merge(products, on='product_id', how='left')
new_customer_items = new_customer_items.merge(translations, on='product_category_name', how='left')

# Step 5: Analyze product categories
category_counts = new_customer_items.groupby('product_category_name_english')['order_id'].count().reset_index()
category_counts.columns = ['category', 'num_orders']
category_counts = category_counts.sort_values('num_orders', ascending=False)

# Step 6: Calculate revenue contribution
new_customer_revenue = new_customer_items.merge(payments, on='order_id', how='left')
revenue_per_category = new_customer_revenue.groupby('product_category_name_english')['payment_value'].sum().reset_index()
revenue_per_category.columns = ['category', 'total_revenue']

final_result = category_counts.merge(revenue_per_category, on='category', how='left')
total_revenue = final_result['total_revenue'].sum()
final_result['revenue_contribution'] = final_result['total_revenue'] / total_revenue * 100

# Step 7: Visualization
# Bar chart for number of orders
plt.figure(figsize=(10, 6))
plt.bar(final_result['category'], final_result['num_orders'])
plt.title('Top Categories by Orders (New Customers)')
plt.xlabel('Category')
plt.ylabel('Number of Orders')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Pie chart for revenue contribution
plt.figure(figsize=(8, 8))
plt.pie(final_result['revenue_contribution'], labels=final_result['category'], autopct='%1.1f%%', startangle=140)
plt.title('Revenue Contribution by Category')
plt.show()

# Step 8: Display the final results
display(final_result)


## Berikut adalah langkah-langkah untuk menjawab **pertanyaan 2**: 

> *Apa 5 wilayah pengiriman dengan waktu pengiriman rata-rata tercepat dan terlambat dalam 6 bulan terakhir, dan bagaimana hal ini memengaruhi tingkat kepuasan pelanggan berdasarkan ulasan?*

---

### **Step 1: Memahami Dataset yang Dibutuhkan**
Dataset yang diperlukan:
1. **`orders_dataset.csv`**
   - Kolom penting:
     - `order_id`
     - `order_purchase_timestamp`
     - `order_delivered_customer_date`
     - `order_approved_at`
   - Untuk menghitung durasi pengiriman.
2. **`geolocation_dataset.csv`**
   - Kolom penting:
     - `geolocation_city`
     - `geolocation_state`
   - Untuk menentukan wilayah pengiriman.
3. **`order_reviews_dataset.csv`**
   - Kolom penting:
     - `order_id`
     - `review_score`
   - Untuk mengukur kepuasan pelanggan.

---

### **Step 2: Data Preparation**
1. **Load Dataset**
   - Import semua dataset yang dibutuhkan ke dalam tools analisis seperti Python (pandas), SQL, atau tools BI seperti Tableau.
   - Contoh dengan Python:
     ```python
     import pandas as pd
     orders = pd.read_csv("orders_dataset.csv")
     geo = pd.read_csv("geolocation_dataset.csv")
     reviews = pd.read_csv("order_reviews_dataset.csv")
     ```

2. **Data Cleaning**
   - Periksa missing values dan data duplikat:
     ```python
     orders.isnull().sum()
     geo.isnull().sum()
     reviews.isnull().sum()
     ```
   - Hapus atau tangani data yang hilang atau tidak valid.
   - Gabungkan dataset sesuai kebutuhan:
     - **`orders` + `reviews`** berdasarkan `order_id`.
     - **`orders` + `geo`** berdasarkan lokasi pengiriman (jika ada informasi lokasi eksplisit dalam orders).

---

### **Step 3: Hitung Waktu Pengiriman**
1. Tambahkan kolom durasi pengiriman (dalam hari):
   ```python
   orders['shipping_duration'] = (
       pd.to_datetime(orders['order_delivered_customer_date']) -
       pd.to_datetime(orders['order_purchase_timestamp'])
   ).dt.days
   ```

2. Filter data untuk 6 bulan terakhir:
   ```python
   recent_orders = orders[
       pd.to_datetime(orders['order_purchase_timestamp']) >= '2024-06-01'
   ]
   ```

---

### **Step 4: Identifikasi Wilayah Pengiriman**
1. Gabungkan data `orders` dengan `geo` untuk mendapatkan informasi wilayah pengiriman:
   ```python
   # Jika ada kolom ID lokasi atau langsung menggunakan kota/provinsi
   orders_geo = recent_orders.merge(
       geo, left_on='customer_city', right_on='geolocation_city', how='left'
   )
   ```

2. Kelompokkan berdasarkan wilayah dan hitung rata-rata durasi pengiriman:
   ```python
   shipping_by_region = orders_geo.groupby(['geolocation_city', 'geolocation_state']).agg({
       'shipping_duration': 'mean'
   }).reset_index()
   ```

---

### **Step 5: Identifikasi Wilayah Pengiriman Tercepat dan Terlambat**
1. Urutkan berdasarkan waktu pengiriman:
   ```python
   fastest_regions = shipping_by_region.sort_values('shipping_duration').head(5)
   slowest_regions = shipping_by_region.sort_values('shipping_duration', ascending=False).head(5)
   ```

2. Tambahkan kolom wilayah yang diidentifikasi tercepat dan terlambat.

---

### **Step 6: Hubungkan dengan Kepuasan Pelanggan**
1. Gabungkan ulasan pelanggan:
   ```python
   reviews_by_region = recent_orders.merge(reviews, on='order_id')
   satisfaction_by_region = reviews_by_region.groupby(['geolocation_city', 'geolocation_state']).agg({
       'review_score': 'mean'
   }).reset_index()
   ```

2. Gabungkan hasil dengan data durasi pengiriman:
   ```python
   final_analysis = shipping_by_region.merge(
       satisfaction_by_region, on=['geolocation_city', 'geolocation_state'], how='left'
   )
   ```

---

### **Step 7: Visualisasi Hasil**
1. **Visualisasi wilayah tercepat dan terlambat**:
   - Gunakan bar chart untuk membandingkan rata-rata waktu pengiriman per wilayah.
   - Contoh dengan matplotlib:
     ```python
     import matplotlib.pyplot as plt
     plt.bar(fastest_regions['geolocation_city'], fastest_regions['shipping_duration'])
     plt.title("5 Wilayah Pengiriman Tercepat")
     plt.xlabel("Kota")
     plt.ylabel("Durasi Pengiriman (Hari)")
     plt.show()
     ```

2. **Visualisasi hubungan waktu pengiriman dan kepuasan pelanggan**:
   - Scatter plot atau heatmap.
   - Contoh:
     ```python
     plt.scatter(final_analysis['shipping_duration'], final_analysis['review_score'])
     plt.title("Hubungan Durasi Pengiriman dan Kepuasan Pelanggan")
     plt.xlabel("Durasi Pengiriman (Hari)")
     plt.ylabel("Review Score")
     plt.show()
     ```

---

### **Step 8: Kesimpulan dan Rekomendasi**
1. Identifikasi wilayah dengan performa logistik baik (tercepat dengan ulasan tinggi) dan buruk (terlambat dengan ulasan rendah).
2. Berikan rekomendasi seperti:
   - Optimalisasi pengiriman untuk wilayah lambat.
   - Promosi di wilayah dengan ulasan baik.

Apakah Anda ingin saya membantu lebih lanjut dengan contoh kode atau visualisasi?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Import datasets
orders = pd.read_csv("orders_dataset.csv")
geo = pd.read_csv("geolocation_dataset.csv")
reviews = pd.read_csv("order_reviews_dataset.csv")

# Step 2: Data Preparation
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])

# Step 3: Calculate shipping duration
orders['shipping_duration'] = (
    orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']
).dt.days

# Filter data for the last 6 months
recent_orders = orders[
    orders['order_purchase_timestamp'] >= '2024-06-01'
]

# Step 4: Combine with geolocation data
# Assuming 'customer_city' exists in orders and matches 'geolocation_city' in geo
geo_orders = recent_orders.merge(
    geo, left_on='customer_city', right_on='geolocation_city', how='left'
)

# Step 5: Analyze shipping by region
shipping_by_region = geo_orders.groupby(['geolocation_city', 'geolocation_state']).agg(
    {'shipping_duration': 'mean'}
).reset_index()
shipping_by_region.columns = ['city', 'state', 'avg_shipping_duration']

# Step 6: Identify fastest and slowest regions
fastest_regions = shipping_by_region.sort_values('avg_shipping_duration').head(5)
slowest_regions = shipping_by_region.sort_values('avg_shipping_duration', ascending=False).head(5)

# Step 7: Combine with customer satisfaction data
reviews_by_region = recent_orders.merge(reviews, on='order_id')
satisfaction_by_region = reviews_by_region.groupby(['customer_city', 'customer_state']).agg(
    {'review_score': 'mean'}
).reset_index()
satisfaction_by_region.columns = ['city', 'state', 'avg_review_score']

final_analysis = shipping_by_region.merge(
    satisfaction_by_region, on=['city', 'state'], how='left'
)

# Step 8: Visualization
# Bar chart for fastest regions
plt.figure(figsize=(10, 6))
plt.bar(fastest_regions['city'], fastest_regions['avg_shipping_duration'])
plt.title("5 Wilayah Pengiriman Tercepat")
plt.xlabel("Kota")
plt.ylabel("Durasi Pengiriman Rata-Rata (Hari)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Bar chart for slowest regions
plt.figure(figsize=(10, 6))
plt.bar(slowest_regions['city'], slowest_regions['avg_shipping_duration'])
plt.title("5 Wilayah Pengiriman Terlambat")
plt.xlabel("Kota")
plt.ylabel("Durasi Pengiriman Rata-Rata (Hari)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Scatter plot for shipping duration vs review score
plt.figure(figsize=(8, 6))
plt.scatter(final_analysis['avg_shipping_duration'], final_analysis['avg_review_score'])
plt.title("Hubungan Durasi Pengiriman dan Kepuasan Pelanggan")
plt.xlabel("Durasi Pengiriman Rata-Rata (Hari)")
plt.ylabel("Skor Review Rata-Rata")
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 9: Display final results
display(final_analysis)


## Berikut adalah langkah-langkah untuk menjawab Pertanyaan 3: 
> "Selama 1 tahun terakhir, kategori produk apa yang memiliki tingkat pengembalian (return rate) tertinggi, dan bagaimana pola ulasan pada produk-produk tersebut?"

---

### **Langkah 1: Memahami Data**
Pastikan Anda memahami struktur dan kolom dalam dataset berikut:
1. **`orders_dataset.csv`**
   - Kolom penting: `order_id`, `order_status`, `order_purchase_timestamp`, `order_approved_at`
   - Status yang relevan: `canceled`, `returned`
2. **`order_items_dataset.csv`**
   - Kolom penting: `order_id`, `product_id`, `seller_id`
3. **`products_dataset.csv`**
   - Kolom penting: `product_id`, `product_category_name`
4. **`product_category_name_translation.csv`**
   - Kolom penting: `product_category_name`, `product_category_name_english`
5. **`order_reviews_dataset.csv`**
   - Kolom penting: `order_id`, `review_score`, `review_comment_message`

---

### **Langkah 2: Preprocessing Data**
1. **Filter Data pada 1 Tahun Terakhir:**
   - Gunakan `order_purchase_timestamp` dari **`orders_dataset.csv`** untuk memfilter pesanan dalam 1 tahun terakhir.

2. **Identifikasi Pengembalian:**
   - Dari **`orders_dataset.csv`**, pilih pesanan dengan status `canceled` atau `returned`.

3. **Gabungkan Data Pesanan dan Produk:**
   - Gabungkan **`orders_dataset.csv`** dan **`order_items_dataset.csv`** berdasarkan `order_id`.
   - Gabungkan hasil tersebut dengan **`products_dataset.csv`** menggunakan `product_id`.

4. **Terjemahkan Kategori Produk:**
   - Gabungkan hasil sebelumnya dengan **`product_category_name_translation.csv`** untuk menerjemahkan kategori produk ke bahasa Inggris.

5. **Gabungkan Data Ulasan:**
   - Gabungkan hasil gabungan di atas dengan **`order_reviews_dataset.csv`** berdasarkan `order_id` untuk menambahkan informasi ulasan.

---

### **Langkah 3: Analisis Tingkat Pengembalian**
1. **Hitung Jumlah Pesanan per Kategori:**
   - Kelompokkan data berdasarkan `product_category_name_english` dan hitung jumlah total pesanan (`total_orders`) untuk setiap kategori.

2. **Hitung Jumlah Pengembalian per Kategori:**
   - Dari data pesanan yang difilter untuk status `canceled` atau `returned`, kelompokkan berdasarkan `product_category_name_english` dan hitung jumlah pengembalian (`returned_orders`) per kategori.

3. **Hitung Tingkat Pengembalian (Return Rate):**
   - Untuk setiap kategori, hitung tingkat pengembalian:
     \[
     \text{return\_rate} = \frac{\text{returned\_orders}}{\text{total\_orders}} \times 100\%
     \]

---

### **Langkah 4: Analisis Pola Ulasan**
1. **Gabungkan Data dengan Ulasan:**
   - Gunakan data ulasan untuk pesanan yang dikembalikan.
   - Analisis kolom `review_score` untuk menghitung rata-rata nilai ulasan (`average_review_score`) dan distribusi ulasan untuk setiap kategori produk.

2. **Temukan Pola:**
   - Identifikasi apakah produk dengan tingkat pengembalian tinggi cenderung memiliki ulasan negatif atau pola tertentu dalam komentar pelanggan.

---

### **Langkah 5: Visualisasi**
1. **Tingkat Pengembalian per Kategori:**
   - Buat grafik batang untuk menunjukkan `return_rate` per kategori produk.
2. **Hubungan dengan Pola Ulasan:**
   - Buat visualisasi tambahan seperti boxplot atau scatter plot untuk menunjukkan hubungan antara `review_score` dan `return_rate`.

---

### **Langkah 6: Insights dan Rekomendasi**
1. Identifikasi kategori produk dengan tingkat pengembalian tertinggi.
2. Jelaskan pola ulasan pada kategori tersebut, misalnya:
   - Apakah pelanggan mengeluhkan kualitas produk, deskripsi yang tidak akurat, atau faktor lain?
3. Rekomendasikan langkah-langkah untuk mengurangi tingkat pengembalian, seperti:
   - Memperbaiki deskripsi produk.
   - Meninjau kualitas produk.
   - Menyediakan panduan penggunaan.

---

Apakah Anda membutuhkan bantuan untuk menerapkan langkah-langkah tersebut, seperti menulis kode untuk memproses data?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Import datasets
orders = pd.read_csv("orders_dataset.csv")
order_items = pd.read_csv("order_items_dataset.csv")
products = pd.read_csv("products_dataset.csv")
translations = pd.read_csv("product_category_name_translation.csv")
reviews = pd.read_csv("order_reviews_dataset.csv")

# Step 2: Filter data for the last year
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
recent_orders = orders[
    orders['order_purchase_timestamp'] >= '2023-12-01'
]

# Step 3: Identify returns
returns = recent_orders[recent_orders['order_status'].isin(['canceled', 'returned'])]

# Step 4: Merge datasets
orders_items_products = order_items.merge(
    products, on='product_id', how='left'
).merge(
    translations, on='product_category_name', how='left'
)
returns_data = returns.merge(orders_items_products, on='order_id', how='left')

# Step 5: Calculate return rates
# Total orders per category
total_orders = orders_items_products.groupby('product_category_name_english').size().reset_index(name='total_orders')

# Returned orders per category
returned_orders = returns_data.groupby('product_category_name_english').size().reset_index(name='returned_orders')

# Merge and calculate return rate
return_rate = total_orders.merge(returned_orders, on='product_category_name_english', how='left')
return_rate['returned_orders'] = return_rate['returned_orders'].fillna(0)
return_rate['return_rate'] = (return_rate['returned_orders'] / return_rate['total_orders']) * 100

# Step 6: Analyze review patterns
returns_reviews = returns_data.merge(reviews, on='order_id', how='left')
reviews_analysis = returns_reviews.groupby('product_category_name_english').agg({
    'review_score': ['mean', 'count'],
    'review_comment_message': 'count'
}).reset_index()
reviews_analysis.columns = ['category', 'avg_review_score', 'review_count', 'comment_count']

# Merge return rate with review analysis
final_analysis = return_rate.merge(reviews_analysis, left_on='product_category_name_english', right_on='category', how='left')

# Step 7: Visualizations
# Bar chart for return rates
plt.figure(figsize=(12, 6))
final_analysis = final_analysis.sort_values('return_rate', ascending=False)
plt.bar(final_analysis['product_category_name_english'], final_analysis['return_rate'])
plt.title("Return Rate by Product Category")
plt.xlabel("Category")
plt.ylabel("Return Rate (%)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Scatter plot for return rate vs. average review score
plt.figure(figsize=(8, 6))
plt.scatter(final_analysis['return_rate'], final_analysis['avg_review_score'])
plt.title("Return Rate vs. Average Review Score")
plt.xlabel("Return Rate (%)")
plt.ylabel("Average Review Score")
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 8: Display final results
display(final_analysis)


## Berikut adalah langkah-langkah sistematis untuk menjawab **Pertanyaan 4**
> *Siapa 10 penjual dengan rata-rata nilai ulasan terbaik dan terburuk selama 12 bulan terakhir, dan bagaimana performa mereka dalam hal jumlah pesanan dan pendapatan?*

---

### **Langkah 1: Memahami Data yang Relevan**
Gunakan dataset berikut:
1. **`order_reviews_dataset.csv`**: Untuk mendapatkan nilai ulasan per pesanan.
2. **`order_items_dataset.csv`**: Untuk menghubungkan ulasan dengan produk dan penjual.
3. **`sellers_dataset.csv`**: Untuk informasi detail penjual.
4. **`orders_dataset.csv`**: Untuk memfilter pesanan dalam 12 bulan terakhir.
5. **`order_payments_dataset.csv`**: Untuk menghitung total pendapatan yang dihasilkan oleh penjual.

---

### **Langkah 2: Filter Data Berdasarkan Waktu**
1. Buka **`orders_dataset.csv`** dan ambil kolom `order_purchase_timestamp`.
2. Filter pesanan yang terjadi dalam 12 bulan terakhir menggunakan:
   - Format tanggal `YYYY-MM-DD`.
   - `order_status` sebagai "delivered" untuk memastikan hanya pesanan yang selesai diproses.

**Output:** Daftar `order_id` untuk pesanan dalam 12 bulan terakhir.

---

### **Langkah 3: Gabungkan Data Ulasan dengan Penjual**
1. Buka **`order_reviews_dataset.csv`** dan ambil kolom `order_id` dan `review_score`.
2. Gabungkan data ini dengan **`order_items_dataset.csv`** menggunakan kolom `order_id` untuk menambahkan informasi:
   - `seller_id`.
   - `price` (harga untuk perhitungan pendapatan).
3. Gabungkan hasil ini dengan **`sellers_dataset.csv`** menggunakan kolom `seller_id` untuk mendapatkan nama penjual atau detail lainnya.

**Output:** Data gabungan dengan informasi:
- `seller_id`, `review_score`, dan `price`.

---

### **Langkah 4: Hitung Statistik Performa Penjual**
1. **Rata-rata Ulasan:**
   - Group by `seller_id` dan hitung rata-rata (`mean`) dari `review_score`.
2. **Jumlah Pesanan:**
   - Hitung jumlah transaksi per penjual (`count` dari `order_id`).
3. **Total Pendapatan:**
   - Hitung total pendapatan (`sum` dari `price`) per penjual.

**Output:** Tabel agregasi berisi:
- `seller_id`, `average_review_score`, `order_count`, dan `total_revenue`.

---

### **Langkah 5: Identifikasi 10 Penjual Terbaik dan Terburuk**
1. Urutkan tabel berdasarkan `average_review_score`:
   - **10 terbaik**: `average_review_score` tertinggi.
   - **10 terburuk**: `average_review_score` terendah.
2. Tampilkan kolom tambahan seperti `order_count` dan `total_revenue` untuk membandingkan performa penjual.

**Output:** Daftar 10 penjual dengan skor ulasan terbaik dan terburuk beserta jumlah pesanan dan total pendapatan.

---

### **Langkah 6: Visualisasi Data**
1. **Distribusi Rata-Rata Ulasan Penjual:**
   - Gunakan histogram atau box plot untuk menunjukkan persebaran nilai ulasan.
2. **Penjual Terbaik vs Terburuk:**
   - Buat bar chart untuk membandingkan:
     - `average_review_score`.
     - `order_count`.
     - `total_revenue` untuk 10 penjual terbaik dan terburuk.

---

### **Langkah 7: Interpretasi Hasil**
1. Analisis faktor-faktor yang menyebabkan perbedaan performa antara penjual terbaik dan terburuk, misalnya:
   - Apakah penjual dengan skor rendah memiliki jumlah pesanan besar namun ulasan buruk?
   - Bagaimana hubungan antara pendapatan dengan skor ulasan?

2. Sampaikan rekomendasi, seperti:
   - Memberikan pelatihan kepada penjual dengan ulasan buruk.
   - Menggandeng penjual terbaik untuk promosi eksklusif.

---

### **Catatan Tambahan**
Jika Anda menggunakan Python atau SQL untuk analisis:
- **Python:** Gunakan pandas untuk manipulasi data dan matplotlib/plotly untuk visualisasi.
- **SQL:** Gabungkan dataset dengan teknik join (`INNER JOIN`, `GROUP BY`, `ORDER BY`).

Saya bisa membantu Anda dengan kode Python atau SQL untuk langkah-langkah ini. Apakah Anda ingin contoh implementasinya?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Step 1: Import datasets
orders = pd.read_csv("orders_dataset.csv")
order_items = pd.read_csv("order_items_dataset.csv")
reviews = pd.read_csv("order_reviews_dataset.csv")
sellers = pd.read_csv("sellers_dataset.csv")
payments = pd.read_csv("order_payments_dataset.csv")

# Step 2: Filter data for the last 12 months
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
recent_orders = orders[(orders['order_purchase_timestamp'] >= '2023-12-01') & (orders['order_status'] == 'delivered')]

# Step 3: Merge data for reviews and sellers
merged_data = recent_orders.merge(
    order_items, on='order_id', how='inner'
).merge(
    reviews, on='order_id', how='left'
).merge(
    sellers, on='seller_id', how='left'
)

# Step 4: Calculate performance metrics
seller_performance = merged_data.groupby('seller_id').agg(
    average_review_score=('review_score', 'mean'),
    order_count=('order_id', 'count'),
    total_revenue=('price', 'sum')
).reset_index()

# Step 5: Identify top and bottom sellers
top_sellers = seller_performance.sort_values('average_review_score', ascending=False).head(10)
bottom_sellers = seller_performance.sort_values('average_review_score', ascending=True).head(10)

# Step 6: Visualization
# Distribution of review scores
plt.figure(figsize=(10, 6))
plt.hist(seller_performance['average_review_score'], bins=20, edgecolor='black')
plt.title("Distribution of Seller Average Review Scores")
plt.xlabel("Average Review Score")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

# Top 10 Sellers by Review Score
plt.figure(figsize=(10, 6))
plt.bar(top_sellers['seller_id'], top_sellers['average_review_score'])
plt.title("Top 10 Sellers by Average Review Score")
plt.xlabel("Seller ID")
plt.ylabel("Average Review Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Bottom 10 Sellers by Review Score
plt.figure(figsize=(10, 6))
plt.bar(bottom_sellers['seller_id'], bottom_sellers['average_review_score'], color='red')
plt.title("Bottom 10 Sellers by Average Review Score")
plt.xlabel("Seller ID")
plt.ylabel("Average Review Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Display final results
display(top_sellers)
display(bottom_sellers)


## Berikut adalah langkah-langkah untuk mengerjakan **Pertanyaan 5**
> *"Metode pembayaran mana yang paling banyak digunakan dalam 6 bulan terakhir, dan bagaimana distribusinya terhadap nilai pesanan di setiap kategori produk?"*

---

### **Step 1: Import dan Praproses Data**
1. **Import Dataset**
   - **Dataset yang diperlukan:**
     - `order_payments_dataset.csv`
     - `orders_dataset.csv`
     - `order_items_dataset.csv`
     - `products_dataset.csv`
     - `product_category_name_translation.csv`
   - Gunakan library seperti **pandas** untuk membaca dataset.

2. **Praproses Data**
   - Bersihkan data dari missing values atau duplikasi.
   - Pastikan kolom yang digunakan (misalnya `order_id`, `payment_type`, `payment_value`, `order_approved_at`, dll.) tidak memiliki inkonsistensi.
   - Konversi kolom tanggal (`order_approved_at`) ke format datetime untuk analisis berdasarkan waktu.

---

### **Step 2: Filter Data untuk 6 Bulan Terakhir**
1. Dari dataset `orders_dataset.csv`, pilih hanya pesanan dengan tanggal persetujuan (`order_approved_at`) dalam 6 bulan terakhir:
   ```python
   import pandas as pd
   from datetime import datetime, timedelta

   orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
   six_months_ago = datetime.now() - timedelta(days=180)
   recent_orders = orders[orders['order_approved_at'] >= six_months_ago]
   ```

2. Gunakan `order_id` dari pesanan ini untuk memfilter dataset terkait lainnya:
   - `order_payments_dataset.csv`
   - `order_items_dataset.csv`

---

### **Step 3: Gabungkan Data**
1. Gabungkan dataset berdasarkan `order_id` untuk mendapatkan hubungan antara metode pembayaran, nilai pembayaran, dan produk:
   - **Gabung `order_payments_dataset` dengan `recent_orders`**:
     ```python
     recent_payments = pd.merge(recent_orders, order_payments, on='order_id')
     ```

   - **Gabung `recent_payments` dengan `order_items_dataset`**:
     ```python
     recent_items = pd.merge(recent_payments, order_items, on='order_id')
     ```

   - **Gabung dengan `products_dataset` untuk mendapatkan kategori produk**:
     ```python
     recent_products = pd.merge(recent_items, products, on='product_id')
     ```

   - **Gabungkan dengan `product_category_name_translation` untuk mendapatkan nama kategori yang diterjemahkan**:
     ```python
     recent_products = pd.merge(
         recent_products, category_translation, on='product_category_name', how='left'
     )
     ```

---

### **Step 4: Analisis Metode Pembayaran**
1. Hitung frekuensi setiap metode pembayaran:
   ```python
   payment_usage = recent_products.groupby('payment_type')['payment_value'].sum().reset_index()
   payment_usage = payment_usage.sort_values(by='payment_value', ascending=False)
   ```

2. Distribusi pembayaran berdasarkan kategori produk:
   ```python
   category_payment = recent_products.groupby(['product_category_name', 'payment_type'])[
       'payment_value'
   ].sum().reset_index()
   ```

---

### **Step 5: Visualisasi Data**
1. **Plot Metode Pembayaran Paling Banyak Digunakan**
   - Gunakan bar chart untuk menunjukkan total nilai pembayaran per metode.
     ```python
     import matplotlib.pyplot as plt

     plt.bar(payment_usage['payment_type'], payment_usage['payment_value'])
     plt.xlabel('Payment Type')
     plt.ylabel('Total Payment Value')
     plt.title('Most Used Payment Methods in the Last 6 Months')
     plt.show()
     ```

2. **Plot Distribusi Pembayaran Berdasarkan Kategori**
   - Gunakan stacked bar chart untuk menunjukkan distribusi metode pembayaran dalam kategori produk.
     ```python
     import seaborn as sns

     sns.barplot(
         data=category_payment,
         x='product_category_name',
         y='payment_value',
         hue='payment_type',
     )
     plt.xticks(rotation=90)
     plt.title('Payment Distribution Across Product Categories')
     plt.show()
     ```

---

### **Step 6: Interpretasi Hasil**
1. Identifikasi metode pembayaran paling populer berdasarkan total nilai pembayaran.
2. Analisis distribusi metode pembayaran per kategori untuk menemukan tren (misalnya, metode tertentu lebih sering digunakan untuk kategori produk tertentu).

---

Jika Anda memerlukan skrip Python untuk langkah-langkah ini, saya dapat membantu menuliskannya lebih detail. Silakan beri tahu!

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Step 1: Load datasets
orders = pd.read_csv("orders_dataset.csv")
payments = pd.read_csv("order_payments_dataset.csv")
order_items = pd.read_csv("order_items_dataset.csv")
products = pd.read_csv("products_dataset.csv")
category_translation = pd.read_csv("product_category_name_translation.csv")

# Step 2: Filter data for the last 6 months
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
six_months_ago = datetime.now() - timedelta(days=180)
recent_orders = orders[orders['order_approved_at'] >= six_months_ago]

# Step 3: Merge datasets
recent_payments = pd.merge(recent_orders, payments, on='order_id')
recent_items = pd.merge(recent_payments, order_items, on='order_id')
recent_products = pd.merge(recent_items, products, on='product_id')
recent_products = pd.merge(
    recent_products, category_translation, on='product_category_name', how='left'
)

# Step 4: Analyze payment methods
# Most used payment methods
payment_usage = recent_products.groupby('payment_type')['payment_value'].sum().reset_index()
payment_usage = payment_usage.sort_values(by='payment_value', ascending=False)

# Payment distribution by product category
category_payment = recent_products.groupby(['product_category_name_english', 'payment_type'])[
    'payment_value'
].sum().reset_index()

# Step 5: Visualizations
# Bar chart for most used payment methods
plt.figure(figsize=(10, 6))
plt.bar(payment_usage['payment_type'], payment_usage['payment_value'])
plt.xlabel('Payment Type')
plt.ylabel('Total Payment Value')
plt.title('Most Used Payment Methods in the Last 6 Months')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Stacked bar chart for payment distribution by category
plt.figure(figsize=(14, 8))
sns.barplot(
    data=category_payment,
    x='product_category_name_english',
    y='payment_value',
    hue='payment_type'
)
plt.xticks(rotation=90)
plt.xlabel('Product Category')
plt.ylabel('Total Payment Value')
plt.title('Payment Distribution Across Product Categories')
plt.legend(title='Payment Type')
plt.tight_layout()
plt.show()

# Display payment usage and category distribution
print("Payment Usage by Method:\n", payment_usage)
print("\nPayment Distribution by Category:\n", category_payment)
